In [20]:
import torch

dense_autoencoder = torch.jit.load('dense_autoencoder_land.pt')
#print(dense_autoencoder)
import pickle
scaler_qv=pickle.load(open('scaler_ocean_qv.pkl','rb'))
import numpy as np
test_tensor= torch.tensor(scaler_qv['mean'][np.newaxis,:],dtype=torch.float32)
print(test_tensor.shape)
y_pred = dense_autoencoder(test_tensor)
y_enc_pred=dense_autoencoder.encoder(test_tensor)
torch.onnx.export(dense_autoencoder.encoder, test_tensor, "GMI_ONNX_Models/dense_encoder_land.onnx", verbose=True, input_names = ['input'], output_names = ['enc_output'])
torch.onnx.export(dense_autoencoder.decoder, y_enc_pred.detach(), "GMI_ONNX_Models/dense_decoder_land.onnx", verbose=True, input_names = ['enc_output'], output_names = ['dec_output'])

dense_autoencoder_ocean = torch.jit.load('dense_autoencoder_ocean.pt')
torch.onnx.export(dense_autoencoder_ocean.encoder, test_tensor, "GMI_ONNX_Models/dense_encoder_ocean.onnx", verbose=True, input_names = ['input'], output_names = ['enc_output'])
torch.onnx.export(dense_autoencoder_ocean.decoder, y_enc_pred.detach(), "GMI_ONNX_Models/dense_decoder_ocean.onnx", verbose=True, input_names = ['enc_output'], output_names = ['dec_output'])

# convert save scaler qv to a npz file
np.savez('scaler_ocean_qv.npz', mean=scaler_qv['mean'], std=scaler_qv['std'])
scaler_qv_land=pickle.load(open('scaler_land_qv.pkl','rb'))
np.savez('scaler_land_qv.npz', mean=scaler_qv_land['mean'], std=scaler_qv_land['std'])


torch.Size([1, 11])
Exported graph: graph(%input : Float(1, 11, strides=[0, 1], requires_grad=0, device=cpu),
      %0.weight : Float(12, 11, strides=[11, 1], requires_grad=0, device=cpu),
      %0.bias : Float(12, strides=[1], requires_grad=0, device=cpu),
      %2.weight : Float(12, 12, strides=[12, 1], requires_grad=0, device=cpu),
      %2.bias : Float(12, strides=[1], requires_grad=0, device=cpu),
      %4.weight : Float(4, 12, strides=[12, 1], requires_grad=0, device=cpu),
      %4.bias : Float(4, strides=[1], requires_grad=0, device=cpu)):
  %/0/Gemm_output_0 : Float(1, 12, strides=[12, 1], device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/0/Gemm"](%input, %0.weight, %0.bias), scope: torch.nn.modules.container.Sequential::/torch.nn.modules.linear.Linear::0 # /Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/nn/modules/linear.py:125:15
  %/1/Relu_output_0 : Float(1, 12, strides=[12, 1], device=cpu) = onnx::Relu[onnx_name="/1/Relu"](%/0/Gemm_output_0), sc

/Users/mgrecu/miniforge3/lib/python3.12/site-packages/torch/onnx/utils.py:780: UserWarning: no signature found for builtin <built-in method __call__ of PyCapsule object at 0x10ea41ad0>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


In [3]:
#print(scaler_qv.keys())
import numpy as np
vars_string=['tc','sfc_type','sk_temp','oe_wvp','near_sfc_precip','xenc','xenc_prec','xenv_enc']
#dict_scaler={"vars":vars_string,"norm_param":norm_param_L}
import pickle
#with open('norm_param_ocean.pkl','wb') as f:
#    pickle.dump(dict_scaler,f)
dict_scaler=pickle.load(open('norm_param_150_ocean.pkl','rb'))
np.savez('GMI_ONNX_Models/norm_param_150_ocean.npz',tc=dict_scaler['norm_param'][0],sfc_type=dict_scaler['norm_param'][1],sk_temp=dict_scaler['norm_param'][2],oe_wvp=dict_scaler['norm_param'][3],near_sfc_precip=dict_scaler['norm_param'][4],xenc=dict_scaler['norm_param'][5],xenc_prec=dict_scaler['norm_param'][6],xenv_enc=dict_scaler['norm_param'][7])
dict_scaler_land=pickle.load(open('norm_param_150_land.pkl','rb'))
np.savez('GMI_ONNX_Models/norm_param_150_land.npz',tc=dict_scaler_land['norm_param'][0],sfc_type=dict_scaler_land['norm_param'][1],sk_temp=dict_scaler_land['norm_param'][2],oe_wvp=dict_scaler_land['norm_param'][3],near_sfc_precip=dict_scaler_land['norm_param'][4],xenc=dict_scaler_land['norm_param'][5],xenc_prec=dict_scaler_land['norm_param'][6],xenv_enc=dict_scaler_land['norm_param'][7])


In [17]:

np.savez('GMI_ONNX_Models/var_string.npz',vars=vars_string)
np.savetxt('GMI_ONNX_Models/var_string.txt',vars_string,fmt='%s')

In [21]:
import onnx

def get_model_io_details(model_path):
    model = onnx.load(model_path)
    graph = model.graph

    def get_details(value_info):
        name = value_info.name
        tensor_type = value_info.type.tensor_type
        shape = [dim.dim_value if dim.HasField("dim_value") else "?" for dim in tensor_type.shape.dim]
        dtype = tensor_type.elem_type
        return {"name": name, "shape": shape, "dtype": dtype}

    inputs = [get_details(input) for input in graph.input]
    outputs = [get_details(output) for output in graph.output]

    return {"inputs": inputs, "outputs": outputs}

# Example usage:
#model_details = get_model_io_details("model.onnx")
#print(model_details)

model_details = get_model_io_details("GMI_ONNX_Models/dense_encoder_land.onnx")
print(model_details)

{'inputs': [{'name': 'input', 'shape': [1, 11], 'dtype': 1}], 'outputs': [{'name': 'enc_output', 'shape': [1, 4], 'dtype': 1}]}


In [7]:
import onnx

# Mapping ONNX TensorProto values to readable dtype names
ONNX_DTYPE_MAP = {
    1: "FLOAT32",
    2: "UINT8",
    3: "INT8",
    4: "UINT16",
    5: "INT16",
    6: "INT32",
    7: "INT64",
    8: "STRING",
    9: "BOOL",
    10: "FLOAT16",
    11: "FLOAT64",
    12: "UINT32",
    13: "UINT64",
    14: "COMPLEX64",
    15: "COMPLEX128",
    16: "BFLOAT16",
}

def get_model_io_details(model_path):
    model = onnx.load(model_path)
    graph = model.graph

    def get_details(value_info):
        name = value_info.name
        tensor_type = value_info.type.tensor_type
        shape = [dim.dim_value if dim.HasField("dim_value") else "?" for dim in tensor_type.shape.dim]
        dtype = ONNX_DTYPE_MAP.get(tensor_type.elem_type, f"Unknown({tensor_type.elem_type})")
        return {"name": name, "shape": shape, "dtype": dtype}

    inputs = [get_details(input) for input in graph.input]
    outputs = [get_details(output) for output in graph.output]

    return {"inputs": inputs, "outputs": outputs}

# Example usage:
#model_details = get_model_io_details("model.onnx")
#print(model_details)
import glob
model_paths=glob.glob("GMI_ONNX_Models/*.onnx")

model_paths=sorted(model_paths)
print(model_paths)

['GMI_ONNX_Models/dense_autoencoder_land.onnx', 'GMI_ONNX_Models/dense_autoencoder_ocean.onnx', 'GMI_ONNX_Models/dense_decoder_land.onnx', 'GMI_ONNX_Models/dense_decoder_ocean.onnx', 'GMI_ONNX_Models/dense_encoder_land.onnx', 'GMI_ONNX_Models/dense_encoder_ocean.onnx', 'GMI_ONNX_Models/land_model.onnx', 'GMI_ONNX_Models/ocean_model.onnx']


In [ ]:

model_details_list=[]
for model_path in model_paths:
    model_details = get_model_io_details(model_path)
    print(model_path,model_details)
    model_details_list.append(model_details)
#model_details = get_model_io_details("GMI_ONNX_Models/dense_encoder_land.onnx")
#print(model_details)

In [8]:
for i,model_details in enumerate(model_details_list):
    print(i,model_paths[i])
    print(model_details)

0 GMI_ONNX_Models/dense_autoencoder_land.onnx
{'inputs': [{'name': 'input', 'shape': [1, 11], 'dtype': 'FLOAT32'}], 'outputs': [{'name': 'output', 'shape': [1, 11], 'dtype': 'FLOAT32'}]}
1 GMI_ONNX_Models/dense_autoencoder_ocean.onnx
{'inputs': [{'name': 'input', 'shape': [1, 11], 'dtype': 'FLOAT32'}], 'outputs': [{'name': 'output', 'shape': [1, 11], 'dtype': 'FLOAT32'}]}
2 GMI_ONNX_Models/dense_decoder_land.onnx
{'inputs': [{'name': 'enc_output', 'shape': [1, 4], 'dtype': 'FLOAT32'}], 'outputs': [{'name': 'dec_output', 'shape': [1, 11], 'dtype': 'FLOAT32'}]}
3 GMI_ONNX_Models/dense_decoder_ocean.onnx
{'inputs': [{'name': 'enc_output', 'shape': [1, 4], 'dtype': 'FLOAT32'}], 'outputs': [{'name': 'dec_output', 'shape': [1, 11], 'dtype': 'FLOAT32'}]}
4 GMI_ONNX_Models/dense_encoder_land.onnx
{'inputs': [{'name': 'input', 'shape': [1, 11], 'dtype': 'FLOAT32'}], 'outputs': [{'name': 'enc_output', 'shape': [1, 4], 'dtype': 'FLOAT32'}]}
5 GMI_ONNX_Models/dense_encoder_ocean.onnx
{'inputs': [{

In [9]:
import numpy as np
f_mod=open("model_details.h","w")
f_mod.write("void models_details_c_(){\n")    
for i,model_details in enumerate(model_details_list):
    stat_path="strcpy(models[%i].model_path,\"%s\");\n"%(i,model_paths[i])
    f_mod.write(stat_path)
    print_path="printf(\"model[%i] =%%s \\n\",models[%i].model_path);\n"%(i,i)
    f_mod.write(print_path)
    for i_in,inputs in enumerate(model_details['inputs']):
        stat_shape="models[%i].inputs[%i].shape_len=%i"%(i,i_in,len(inputs['shape']))
        stat_name="strcpy(models[%i].inputs[%i].name,\"%s\")"%(i,i_in,inputs['name'])
        stat_nelem="models[%i].inputs[%i].n_elem=%i"%(i,i_in,np.prod(inputs['shape']))
        print(stat_name)
        f_mod.write(stat_name+';\n')
        print(stat_shape)
        f_mod.write(stat_shape+';\n')
        f_mod.write(stat_nelem+';\n')
        total_size=1
        for k in range(len(inputs['shape'])):
            stat_shape_def="models[%i].inputs[%i].shape[%i]=%i"%(i,i_in,k,inputs['shape'][k])
            total_size*=inputs['shape'][k]
            print(stat_shape_def)
            f_mod.write(stat_shape_def+';\n')
        if inputs['dtype']=='FLOAT32':
            stat_type="models[%i].inputs[%i].dtype=1"%(i,i_in)
        print(stat_type)
        f_mod.write(stat_type+';\n')
        #allocate input tensor
        stat_alloc="models[%i].input_data[%i]=(float *) malloc(sizeof(float)*%i)"%(i,i_in,total_size)
        print(stat_alloc)
        f_mod.write(stat_alloc+';\n')
    stat_num_inputs="models[%i].num_inputs=%i"%(i,len(model_details['inputs']))
    f_mod.write(stat_num_inputs+';\n')
    stat_num_outputs="models[%i].num_outputs=%i"%(i,len(model_details['outputs']))
    f_mod.write(stat_num_outputs+';\n')
    for i_out,outputs in enumerate(model_details['outputs']):
        stat_shape="models[%i].outputs[%i].shape_len=%i"%(i,i_out,len(outputs['shape']))
        stat_name="strcpy(models[%i].outputs[%i].name,\"%s\")"%(i,i_out,outputs['name'])
        stat_nelem="models[%i].outputs[%i].n_elem=%i"%(i,i_out,np.prod(outputs['shape']))
        print(stat_name)
        f_mod.write(stat_name+';\n')
        print(stat_shape)
        f_mod.write(stat_shape+';\n')
        f_mod.write(stat_nelem+';\n')
        total_size=1
        for k in range(len(outputs['shape'])):
            stat_shape_def="models[%i].outputs[%i].shape[%i]=%i"%(i,i_out,k,outputs['shape'][k])
            total_size*=outputs['shape'][k]
            print(stat_shape_def)
            f_mod.write(stat_shape_def+';\n')
        if outputs['dtype']=='FLOAT32':
            stat_type="models[%i].outputs[%i].dtype=1"%(i,i_out)
        print(stat_type)
        f_mod.write(stat_type+';\n')
        stat_alloc="models[%i].output_data[%i]=(float *) malloc(sizeof(float)*%i)"%(i,i_out,total_size)
        print(stat_alloc)
        f_mod.write(stat_alloc+';\n')

f_mod.write("}\n")
f_mod.close()
        #print(inputs['name'],inputs['shape'],inputs['dtype'])
#for outputs in model_details['outputs']:
#    print(outputs['name'],outputs['shape'],outputs['dtype'])

strcpy(models[0].inputs[0].name,"input")
models[0].inputs[0].shape_len=2
models[0].inputs[0].shape[0]=1
models[0].inputs[0].shape[1]=11
models[0].inputs[0].dtype=1
models[0].input_data[0]=(float *) malloc(sizeof(float)*11)
strcpy(models[0].outputs[0].name,"output")
models[0].outputs[0].shape_len=2
models[0].outputs[0].shape[0]=1
models[0].outputs[0].shape[1]=11
models[0].outputs[0].dtype=1
models[0].output_data[0]=(float *) malloc(sizeof(float)*11)
strcpy(models[1].inputs[0].name,"input")
models[1].inputs[0].shape_len=2
models[1].inputs[0].shape[0]=1
models[1].inputs[0].shape[1]=11
models[1].inputs[0].dtype=1
models[1].input_data[0]=(float *) malloc(sizeof(float)*11)
strcpy(models[1].outputs[0].name,"output")
models[1].outputs[0].shape_len=2
models[1].outputs[0].shape[0]=1
models[1].outputs[0].shape[1]=11
models[1].outputs[0].dtype=1
models[1].output_data[0]=(float *) malloc(sizeof(float)*11)
strcpy(models[2].inputs[0].name,"enc_output")
models[2].inputs[0].shape_len=2
models[2].inputs

In [13]:
print(inputs['shape'][0])

1


Could you extend the code below to be more general and work with multiple models defined using information of the type 
GMI_ONNX_Models/land_model.onnx {'inputs': [{'name': 'input', 'shape': [1, 19, 128, 48], 'dtype': 'FLOAT32'}], 'outputs': [{'name': 'output_rec', 'shape': [1, 13, 128, 48], 'dtype': 'FLOAT32'}, {'name': 'output_pred', 'shape': [1, 12, 128, 48], 'dtype': 'FLOAT32'}]}?

The input and ouput information could be stored in simple C structures emulating python general lists.


#include <stdio.h>
#include <stdlib.h>
#include <onnxruntime_c_api.h>

// Global variables for the ONNX Runtime environment, session, and input/output tensors
OrtEnv* env;
OrtSession* session;
OrtSessionOptions* session_options;
//OrtAllocator* allocator;
OrtValue* input_tensor;
OrtValue* output_tensor;
OrtMemoryInfo* memory_info;
OrtStatus *status;
// Initialization function
OrtApi *g_api[4];


void init_onnx_runtime_(void) {
    // Initialize the ONNX Runtime environment
    const OrtApi *api = OrtGetApiBase()->GetApi(ORT_API_VERSION);
    //"conv_and_strat_model_kuka_jan24_2025.onnx"
    for(int im=0;im<4;im++)
      {
	char model_path[150];
	if(im==0)
	  strcpy(model_path,"onnx_models/conv_and_strat_model_ku_feb4_2025_01.onnx");
	if(im==1)
	  strcpy(model_path,"onnx_models/conv_ku_feb4_2025_01.onnx");
	if(im==2)
	  strcpy(model_path,"onnx_models/strat_ku_ocean_feb4_2025_01.onnx");
	if(im==3)
	  strcpy(model_path,"onnx_models/conv_ku_ocean_feb4_2025_01.onnx");
	g_api[im] = (OrtApi *)api;
	status = g_api[im]->CreateEnv(ORT_LOGGING_LEVEL_WARNING, "test", &env);
	status = g_api[im]->CreateSessionOptions(&session_options);
	status = g_api[im]->SetIntraOpNumThreads(session_options, 1);
	status = g_api[im]->CreateSession(env, model_path, session_options, &session);
	if (status != NULL) {
	  const char* msg = g_api[im]->GetErrorMessage(status);
	  fprintf(stderr, "Failed to run ONNX model: %s\n", msg);
	  g_api[im]->ReleaseStatus(status);
	}
	status = g_api[im]->CreateCpuMemoryInfo(OrtArenaAllocator, OrtMemTypeDefault, &memory_info);
      }

}

void call_onnx_(float *input_data, int *lengths_data, float *output_data, int *batch_size, int *seq_len, int *input_size, int *output_size, int *im) {
    //'input': input_data, 'n_seq'
    const char* input_names[] = {"input", "n_seq"};
    const char* output_names[] = {"output"};

    int64_t input_shape[3] = {(int64_t)(*batch_size), (int64_t)(*seq_len), (int64_t)(*input_size)};
    size_t input_tensor_size = (*batch_size) * (*seq_len) * (*input_size);
    OrtValue* input_tensor = NULL;

    status=g_api[*im]->CreateTensorWithDataAsOrtValue(
        memory_info, input_data, input_tensor_size * sizeof(float),
        input_shape, 3, ONNX_TENSOR_ELEMENT_DATA_TYPE_FLOAT, &input_tensor);
    //printf("batch_size: %d\n", *batch_size);
    //printf("seq_len: %d\n", *seq_len);
    //printf("input_size: %d\n", *input_size);
    //printf("output_size: %d\n", *output_size);
    // Prepare seq_lengths tensor
    int64_t lengths_shape[1] = {(int32_t)(*batch_size)};
    OrtValue* lengths_tensor = NULL;
    int32_t *lengths_data_t;
    lengths_data_t = (int32_t *)malloc((*batch_size) * sizeof(int32_t));
    for (int i = 0; i < *batch_size; i++) {
        lengths_data_t[i] = (int32_t)lengths_data[i];
    }
    status=g_api[*im]->CreateTensorWithDataAsOrtValue(
        memory_info, lengths_data_t, (*batch_size) * sizeof(int32_t),
        lengths_shape, 1, ONNX_TENSOR_ELEMENT_DATA_TYPE_INT32, &lengths_tensor);
    
    if (status != NULL) {
        const char* msg = g_api[*im]->GetErrorMessage(status);
        fprintf(stderr, "Failed to run ONNX model: %s\n", msg);
        fprintf(stderr, "batch_size: %d\n", *batch_size);
        g_api[*im]->ReleaseStatus(status);
    }
    // Create input names and tensors
    const OrtValue* input_tensors[] = {input_tensor, lengths_tensor};

    // Prepare output tensor
    int64_t output_shape[3] = {(int64_t)(*batch_size), (int64_t)(*seq_len), (int64_t)(*output_size)};
    size_t output_tensor_size = (*batch_size) * (*seq_len) * (*output_size);
    OrtValue* output_tensor = NULL;

    status=g_api[*im]->CreateTensorWithDataAsOrtValue(
        memory_info, output_data, output_tensor_size * sizeof(float),
        output_shape, 3, ONNX_TENSOR_ELEMENT_DATA_TYPE_FLOAT, &output_tensor);

    
    // Run the model
    status = g_api[*im]->Run(
        session, NULL, input_names, input_tensors, 2,
        output_names, 1, &output_tensor);

    if (status != NULL) {
        const char* msg = g_api[*im]->GetErrorMessage(status);
        fprintf(stderr, "Failed to run ONNX model: %s\n", msg);
        g_api[*im]->ReleaseStatus(status);
    }

    // Release resources
    g_api[*im]->ReleaseValue(input_tensor);
    g_api[*im]->ReleaseValue(lengths_tensor);
    g_api[*im]->ReleaseValue(output_tensor);
    free(lengths_data_t);
}
GMI_ONNX_Models/land_model.onnx {'inputs': [{'name': 'input', 'shape': [1, 19, 128, 48], 'dtype': 'FLOAT32'}], 'outputs': [{'name': 'output_rec', 'shape': [1, 13, 128, 48], 'dtype': 'FLOAT32'}, {'name': 'output_pred', 'shape': [1, 12, 128, 48], 'dtype': 'FLOAT32'}]}
